<a href="https://colab.research.google.com/github/frendev/Capstone/blob/master/week5_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Cafe Mumbai**
- Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open cafe
***
### 1. Import libraries

In [2]:
pip install geocoder


     |████████████████████████████████| 102kB 5.9MB/s 


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [0]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [0]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [0]:
# create a list to store neighborhood data
neighborhoodList = []


In [0]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)



In [8]:
# create a new DataFrame from the list
pu_df = pd.DataFrame({"Neighborhood": neighborhoodList})

pu_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [9]:
# print the number of rows of the dataframe
pu_df.shape

(42, 1)

### 3. Get the geographical coordinates

In [0]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in pu_df["Neighborhood"].tolist() ]

In [12]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00426978951916, 72.85579215153318],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21094000000005, 72.84137000000004],
 [19.048530000000028, 72.93222000000003

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
# merge the coordinates into the original dataframe
pu_df['Latitude'] = df_coords['Latitude']
pu_df['Longitude'] = df_coords['Longitude']

In [100]:

# check the neighborhoods and the coordinates
print(pu_df.shape)
pu_df


(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [0]:
# save the DataFrame as CSV file
pu_df.to_csv("pu_df.csv", index=False)

In [0]:
from google.colab import files
files.download('pu_df.csv') 

### 4. Create a map of Pune with neighborhoods superimposed on top

In [17]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [103]:
# create map of Toronto using latitude and longitude values
map_pu = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(pu_df['Latitude'], pu_df['Longitude'], pu_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.1).add_to(map_pu)  
    
map_pu

In [0]:
# save the map as HTML file
map_pu.save('map_pu.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = 'OIHBMD2OABLTV3OSBHCAYYQUV0IHZR4FVS3DMCLML50BHUDM' # your Foursquare ID
CLIENT_SECRET = 'RSQPDWI2ZJ4W5HRDSFDCM1NF4YXKZXJGQADUIDHZWYLYAB3H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OIHBMD2OABLTV3OSBHCAYYQUV0IHZR4FVS3DMCLML50BHUDM
CLIENT_SECRET:RSQPDWI2ZJ4W5HRDSFDCM1NF4YXKZXJGQADUIDHZWYLYAB3H


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [151]:
radius = 2000
LIMIT = 5
res=[]
venues = []
data=[]
for lat, long, neighborhood in zip(pu_df['Latitude'],pu_df['Longitude'], pu_df['Neighborhood']):
    print(neighborhood)
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
     # make the GET request
    data.append(requests.get(url).json())
    for d in data:
      if(d['response']['groups'][0]['items']):
        res.append(d['response']['groups'][0]['items'])
    for r1 in res:
      for r2 in r1:
        venues.append((neighborhood,lat,long,r2['venue']['name'], 
            r2['venue']['location']['lat'], 
            r2['venue']['location']['lng'],  
            r2['venue']['categories'][0]['name']))
  

    
    

Andheri
Anushakti Nagar
Baiganwadi
Bandra
Bhandup
Borivali
Charkop
Chembur
Dahisar
Devipada
Dombivli
Eastern Suburbs (Mumbai)
Ghatkopar
Goregaon
Grant Road
Jogeshwari
Juhu
Kalyan
Kandivali
Kanjurmarg
Kausa
Kurla
Mahavir Nagar (Kandivali)
Mankhurd
Matharpacady, Mumbai
Mira Road
Mogra Village
Mulund
Mumbra
Pestom sagar
Seven Bungalows
Shil Phata
Sion, Mumbai
Sonapur, Bhandup
Thakur village
Tilak Nagar (Mumbai)
Uttan
Vashi
Vikhroli
Wadala
Western Suburbs (Mumbai)
Worli


In [153]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(65435, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
3,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


**Let's check how many venues were returned for each neighorhood**

In [154]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,5,5,5,5,5,5
Anushakti Nagar,15,15,15,15,15,15
Baiganwadi,30,30,30,30,30,30
Bandra,50,50,50,50,50,50
Bhandup,75,75,75,75,75,75
Borivali,105,105,105,105,105,105
Charkop,140,140,140,140,140,140
Chembur,180,180,180,180,180,180
Dahisar,225,225,225,225,225,225


**Let's find out how many unique categories can be curated from all the returned venues**

In [155]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 70 uniques categories.


In [156]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Falafel Restaurant',
       'Ice Cream Shop', 'Food', 'Chinese Restaurant', 'Pub',
       'Bus Station', 'Plaza', 'Fried Chicken Joint', 'Food & Drink Shop',
       'Platform', 'Hot Dog Joint', 'Lounge', 'Café', 'Dessert Shop',
       'Big Box Store', 'Coffee Shop', 'Food Truck',
       'Fast Food Restaurant', 'Scenic Lookout', 'Bar',
       'Asian Restaurant', 'Snack Place', 'Vegetarian / Vegan Restaurant',
       'Juice Bar', 'Train Station', 'Seafood Restaurant',
       'Historic Site', 'Indie Movie Theater', 'Arcade', 'Cupcake Shop',
       'Hotel', 'Garden', 'Roof Deck', 'Donut Shop',
       'Gym / Fitness Center', 'Restaurant', 'Bookstore', 'BBQ Joint',
       'Mughlai Restaurant', "Women's Store", 'Playground',
       'Dance Studio', 'Pizza Place', 'Cheese Shop',
       'General College & University', 'Convenience Store',
       'Mexican Restaurant', 'Sculpture Garden'], dtype=object)

In [157]:

"Café" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [175]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(65435, 71)


,Neighborhoods,ATM,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Big Box Store,Bookstore,Breakfast Spot,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Coffee Shop,Convenience Store,Cupcake Shop,Dance Studio,Dessert Shop,Diner,Donut Shop,Falafel Restaurant,Fast Food Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Gaming Cafe,Garden,General College & University,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Juice Bar,Lounge,Maharashtrian Restaurant,Mexican Restaurant,Mughlai Restaurant,Multiplex,Neighborhood,Pizza Place,Platform,Playground,Plaza,Pub,Restaurant,Roof Deck,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Tea Room,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Andheri,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [176]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(42, 71)


,Neighborhoods,ATM,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Big Box Store,Bookstore,Breakfast Spot,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Coffee Shop,Convenience Store,Cupcake Shop,Dance Studio,Dessert Shop,Diner,Donut Shop,Falafel Restaurant,Fast Food Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Gaming Cafe,Garden,General College & University,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Juice Bar,Lounge,Maharashtrian Restaurant,Mexican Restaurant,Mughlai Restaurant,Multiplex,Neighborhood,Pizza Place,Platform,Playground,Plaza,Pub,Restaurant,Roof Deck,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Tea Room,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Andheri,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Anushakti Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.266667,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Baiganwadi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.066667,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.200000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bandra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.020000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.080000,0.000000,0.060000,0.040000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.160000,0.160000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.060000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bhandup,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.053333,0.026667,0.000000,0.053333,0.013333,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.066667,0.000000,0.053333,0.040000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.146667,0.173333,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [177]:
len(kl_grouped[kl_grouped["Café"] > 0])

39

**Create a new DataFrame for Cafes data only**

In [0]:
mu_cafe = kl_grouped[["Neighborhoods","Café"]]

In [179]:
mu_cafe.head()

,Neighborhoods,Café
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.020000
4,Bhandup,0.026667


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [180]:
# set number of clusters
kclusters = 3

kl_clustering = mu_cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = mu_cafe.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [182]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Café,Cluster Labels
0,Andheri,0.000000,2
1,Anushakti Nagar,0.000000,2
2,Baiganwadi,0.000000,2
3,Bandra,0.020000,1
4,Bhandup,0.026667,1


In [183]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(pu_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(42, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,2,19.118459,72.841763
1,Anushakti Nagar,0.000000,2,19.042830,72.927340
2,Baiganwadi,0.000000,2,19.062940,72.926630
3,Bandra,0.020000,1,19.054370,72.840170
4,Bhandup,0.026667,1,19.145560,72.948560


In [184]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(42, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
41,Worli,0.041620,0,19.007440,72.816880
23,Mankhurd,0.034667,0,19.048530,72.932220
24,"Matharpacady, Mumbai",0.035692,0,18.950694,72.827268
25,Mira Road,0.036467,0,19.265674,72.870681
26,Mogra Village,0.037135,0,24.375900,75.954570
27,Mulund,0.037624,0,19.171830,72.955650
28,Mumbra,0.037981,0,19.167632,73.021408
29,Pestom sagar,0.038211,0,19.070640,72.902170
31,Shil Phata,0.039140,0,19.146580,73.040050
30,Seven Bungalows,0.038744,0,19.131342,72.816342


**Finally, let's visualize the resulting clusters**

In [185]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [130]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
20,Kausa,0.077922,0,19.127580,72.825390
23,Mankhurd,0.070000,0,19.048530,72.932220
24,"Matharpacady, Mumbai",0.067692,0,18.950694,72.827268
25,Mira Road,0.065527,0,19.265674,72.870681
26,Mogra Village,0.063660,0,24.375900,75.954570
27,Mulund,0.061881,0,19.171830,72.955650
28,Mumbra,0.060185,0,19.167632,73.021408
29,Pestom sagar,0.058568,0,19.070640,72.902170
30,Seven Bungalows,0.057026,0,19.131342,72.816342
31,Shil Phata,0.055556,0,19.146580,73.040050


#### Cluster 1

In [131]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
3,Bandra,0.100000,1,19.054370,72.840170
4,Bhandup,0.133333,1,19.145560,72.948560
5,Borivali,0.142857,1,19.229360,72.857510
6,Charkop,0.142857,1,19.208660,72.826120
7,Chembur,0.138889,1,19.062180,72.902410
8,Dahisar,0.133333,1,19.250030,72.859070
9,Devipada,0.127273,1,19.224690,72.866050
11,Eastern Suburbs (Mumbai),0.115385,1,19.004270,72.855792
12,Ghatkopar,0.109890,1,19.086523,72.909008
13,Goregaon,0.104762,1,19.164550,72.849460


#### Cluster 2

In [132]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
2,Baiganwadi,0.0,2,19.062940,72.926630
1,Anushakti Nagar,0.0,2,19.042830,72.927340
0,Andheri,0.0,2,19.118459,72.841763


#### Observations:
As we can see, most of the cafes are concentrated in cluster 1 with highest number. Cluster 0 comprises of moderate number of cafes. Cluster 2,On the other hand,has very low number of or no cafes at all. This represents a great opportunity and high potential areas to open new cafes as there is very little to no competition from existing cafes. Also, with unique ideas or strategies, there is a chance of growth in cluster 0 places, but the new cafes will definitely face moderate competition from other cafes.Lastly, investors and new businesses are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.